In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


url = f'https://www.fantasypros.com/nfl/projections/qb.php?scoring=PPR'
tables = pd.read_html(url)
table = tables[0]

all_player_fantasy_points = {}

In [ ]:
table.info()

In [3]:
table_column_mapper = {'qb':['Player','Pass Att','Pass Comp','Pass Yds','Pass TDs','Ints','Rush Att','Rush Yds','Rush TDs','Fumbles','Fantasy'],'rb':['Player','Rush Att', 'Rush Yds','Rush TDs', 'Receptions','Rec Yds','Rec TDs','Fumbles','Fantasy'],'wr':['Player','Receptions','Rec Yds','Rec TDs','Rush Att', 'Rush Yds','Rush TDs','Fumbles','Fantasy'],'te':['Player','Receptions','Rec Yds','Rec TDs','Fumbles','Fantasy'],'k':['Player','Field Goal','Field Goal Att','Extra Point','Fantasy'],'dst':['Team','Sacks','Interceptions','Fumbles Recovered','Forced Fumbles','Touchdowns','Safety','Points Against','Yards Against','Fantasy']}

In [ ]:
table.head()

In [2]:
def name_format(player_name):
    trash_characters = [".", "-", "'", "Jr", "IIII", "III", "II"]
    player_name = player_name.split(' ')
    player_name = ' '.join(player_name[:-1])
    for char in trash_characters:
        player_name = player_name.replace(char, "")
    return player_name.strip()

name_format('Josh Allen BUF')

'Josh Allen'

In [ ]:
table.Player = table.Player.apply(name_format)

In [ ]:
player_url = 'https://www.pro-football-reference.com/players/C/CookDa01.htm'
player_tables = pd.read_html(player_url)
player_table = player_tables[0]
print(player_table)

In [ ]:
player_table.info()

In [ ]:
player_table = player_table[[('Unnamed: 0_level_0', 'Year'),('Unnamed: 1_level_0', 'Age'),('Unnamed: 2_level_0', 'Tm'),('Unnamed: 3_level_0', 'Pos'),('Unnamed: 4_level_0', 'No.'),('Games', 'G'),('Games', 'GS'),('Rushing', 'Rush'),('Rushing', 'Yds'),('Rushing', 'TD'),('Rushing', '1D'),('Receiving', 'Tgt'),('Receiving', 'Rec'),('Receiving', 'Yds'),('Receiving', 'TD'),('Receiving', '1D'),('Receiving', 'Ctch%'),('Total Yds', 'Touch'),('Unnamed: 30_level_0', 'Fmb')]]

player_table.columns = ['Year','Age','Team','Pos','Number','Games_Played','Games_Started','Rush_Att','Rush_Yds','Rush_Td','Rushing_First_Down','Receptions','Receiving_Targets','Receiving_Yds','Receiving_Td','Receiving_First_Down','Receiving_Percentage','Touches','Fumbles']

In [ ]:
def remove_spec_chars(word):
    SPECIAL_CHARACTERS = [
    "/",
    "\\",
    "",
    "'",
    ",",
    ".",
    "!",
    "@",
    "#",
    "$",
    "%",
    "^",
    "&",
    "*",
    "(",
    ")",
    "-",
    "_",
    "=",
    "+",
    "[",
    "]",
    "{",
    "}",
    "|",
    ":",
    ";",
    "<",
    ">",
    "?",
    "`",
    "~",
    ]
    for char in SPECIAL_CHARACTERS:
        if char in word:
            word = word.replace(char,'')
    return word


career_index = player_table[player_table.Year == 'Career'].index
player_table = player_table.drop(index=career_index)
player_table.Pos = player_table.Pos.apply(str.upper)
player_table.Year = player_table.Year.apply(remove_spec_chars)
player_table.Number = player_table.Number.apply(int)
player_table.Receiving_Percentage = player_table.Receiving_Percentage.apply(remove_spec_chars).apply(float)/1000



In [ ]:
player_table.Receiving_Percentage

In [ ]:
def calculate_fantasy_points(blk_kick=0,def_kr_td=0,def_pr_td=0,def_st_ff=0,def_st_fum_rec=0,def_st_td=0,def_td=0,ff=0,fgm_0_19=0,fgm_20_29=0,fgm_30_39=0,fgm_40_49=0,fgm_50p=0,fgmiss=0,fum=0,fum_lost=0,fum_rec=0,fum_rec_td=0,def_int=0,kr_td=0,pass_2pt=0,pass_int=0,pass_td=0,pass_yd=0,pr_td=0,pts_allow_0=0,pts_allow_1_6=0,pts_allow_7_13=0,pts_allow_14_20=0,pts_allow_21_27=0,pts_allow_28_34=0,pts_allow_35p=0,rec=0,rec_2pt=0,rec_td=0,rec_yd=0,rush_2pt=0,rush_td=0,rush_yd=0,sack=0,safe=0,st_ff=0,st_fum_rec=0,st_td=0,xpm=0,xpmiss=0):
    score = 0
    score += blk_kick * 2
    score += def_kr_td * 0
    score += def_pr_td * 0
    score += def_st_ff * 0
    score += def_st_fum_rec * 2
    score += def_st_td * 6
    score += def_td * 6
    score += ff * 1
    score += fgm_0_19 * 3
    score += fgm_20_29 * 3
    score += fgm_30_39 * 3
    score += fgm_40_49 * 4
    score += fgm_50p * 5
    score += fgmiss * -1
    score += fum * 0
    score += fum_lost * -2
    score += fum_rec * 2
    score += fum_rec_td * 6
    score += def_int * 2
    score += kr_td * 6
    score += pass_2pt * 2
    score += pass_int * -2
    score += pass_td * 4
    score += pass_yd * 0.03999999910593033
    score += pr_td * 6
    score += pts_allow_0 * 10
    score += pts_allow_1_6 * 7
    score += pts_allow_7_13 * 4
    score += pts_allow_14_20 * 1
    score += pts_allow_21_27 * 0
    score += pts_allow_28_34 * -1
    score += pts_allow_35p * -4
    score += rec * 1
    score += rec_2pt * 2
    score += rec_td * 6
    score += rec_yd * 0.10000000149011612
    score += rush_2pt * 2
    score += rush_td * 6
    score += rush_yd * 0.10000000149011612
    score += sack * 1
    score += safe * 2
    score += st_ff * 0
    score += st_fum_rec * 2
    score += st_td * 6
    score += xpm * 1
    score += xpmiss * -1
    return score

    

In [3]:

def get_fantasy_stats(week):
    positions = ['qb','wr','te','rb']
    for pos in positions:
        url = f'https://www.fantasypros.com/nfl/projections/{pos}.php?scoring=PPR&week={week}'
        data = requests.get(url)
        data_content = data.content
        soup = BeautifulSoup(data_content,'html.parser')
        fantasy_table = soup.find(id='data')
        fantasy_table = fantasy_table.find('tbody')
        fantasy_table_rows = fantasy_table.find_all('tr')
        for player_row in fantasy_table_rows:
            #print(player_row)
            player_name = player_row.find(class_='player-name').text
            print(player_name)
            player_name = name_format(player_name)
            fantasy_points = player_row.find_all('td')[-1]['data-sort-value']
            all_player_fantasy_points[player_name] = fantasy_points



In [6]:
def get_fantasy_stats_df(week):
    fantasy_table = pd.DataFrame()
    positions = ['dst','qb','wr','te','rb','k']
    for pos in positions:
        url = f'https://www.fantasypros.com/nfl/projections/{pos}.php?scoring=PPR&week={week}'
        data = pd.read_html(url)
        pos_table = data[0]
        pos_table.columns = table_column_mapper[pos]
        print(pos_table)
        pos_table = pos_table[[('Unnamed: 0_level_0', 'Player'), ('MISC', 'FPTS')]]
        pos_table.columns = ['Player','Fantasy_Points']
        pos_table.Player = pos_table.Player.apply(name_format)
        break
        
        return fantasy_table

In [7]:
get_fantasy_stats_df('draft')
#fantasy_table = get_fantasy_stats_df('draft')

                      Player  SACK   INT    FR    FF   TD  SAFETY     PA  \
0       Tampa Bay Buccaneers  46.4  15.4  11.2  17.4  2.8     1.0  371.7   
1        Pittsburgh Steelers  47.8  14.7  10.3  15.8  2.7     1.0  365.4   
2           Los Angeles Rams  47.2  14.1  10.7  15.7  2.7     1.0  367.1   
3         Indianapolis Colts  42.7  13.8  10.7  17.2  2.9     1.0  387.6   
4   Washington Football Team  47.3  13.7  10.2  15.6  2.4     0.5  372.6   
5        Philadelphia Eagles  45.9  12.4  10.2  16.4  2.5     1.0  411.3   
6           Baltimore Ravens  38.2  13.0  11.3  18.0  3.2     1.0  354.8   
7         New Orleans Saints  43.8  14.5   9.3  13.1  2.5     0.5  384.5   
8           Seattle Seahawks  42.1  13.9  10.0  14.6  2.4     1.5  394.2   
9          Arizona Cardinals  46.7  11.6  10.4  17.0  2.1     1.0  391.4   
10      New England Patriots  39.9  15.4   8.8  12.6  2.6     0.5  359.2   
11         Carolina Panthers  36.9  13.6  12.3  20.3  2.5     0.5  427.7   
12         G

KeyError: "None of [Index([('Unnamed: 0_level_0', 'Player'), ('MISC', 'FPTS')], dtype='object')] are in the [columns]"

In [46]:
fantasy_table

,Player,Fantasy_Points
0,Josh Allen,380.1
1,Patrick Mahomes,377.4
2,Kyler Murray,358.9
3,Lamar Jackson,348.1
4,Dak Prescott,341.7
...,...,...
91,Nate Sudfeld,0.0
92,Will Grier,0.0
93,Josh Johnson,0.0
94,Nathan Peterman,0.0


In [31]:
fantasy_table.columns = ['Player','Fantasy_Points']
fantasy_table

,Player,Fantasy_Points
0,Josh Allen BUF,380.1
1,Patrick Mahomes II KC,377.4
2,Kyler Murray ARI,358.9
3,Lamar Jackson BAL,348.1
4,Dak Prescott DAL,341.7
...,...,...
91,Nate Sudfeld SF,0.0
92,Will Grier CAR,0.0
93,Josh Johnson NYJ,0.0
94,Nathan Peterman LV,0.0


In [32]:
fantasy_table['Player'] = fantasy_table['Player'].apply(name_format)
fantasy_table

/Users/rileymprice/Documents/GitHub/fantasy_football_predictions/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Player,Fantasy_Points
0,Josh Allen,380.1
1,Patrick Mahomes,377.4
2,Kyler Murray,358.9
3,Lamar Jackson,348.1
4,Dak Prescott,341.7
...,...,...
91,Nate Sudfeld,0.0
92,Will Grier,0.0
93,Josh Johnson,0.0
94,Nathan Peterman,0.0
